## libs

In [1]:
# import openpyxl as ox
# from openpyxl.worksheet.table import Table, TableStyleInfo
import os
import pandas as pd
import numpy as np
from pathlib import Path
import xlwings as xw
import sqlite3 as sql

#from openpyxl import Workbook

## database - get data

In [3]:
db_path="D:/04_edu/04. Data/SQL_sample_database/"
db_file="northwind.db"

In [4]:
db_path+db_file

'D:/04_edu/04. Data/SQL_sample_database/northwind.db'

In [5]:
qry_1="""select  
strftime('%Y.%m', OrderDate) okres
,count(*) cnt
from   Orders
group by 1
order by 1"""

qry_2="""select  
strftime('%Y.%m', OrderDate) okres
,cast(strftime('%Y', OrderDate) as int) rok
,cast(strftime('%m', OrderDate) as int) okres
,count(*) cnt
from   Orders
group by 1,2,3
order by 1"""

qry_3="""select  
strftime('%Y.%m', OrderDate) okres
,cast(strftime('%Y', OrderDate) as int) rok
,cast(strftime('%m', OrderDate) as int) okres
,count(distinct a.OrderID) cnt_orders
,sum(Quantity) products_units
from   Orders a
join [Order Details] b
on a.OrderID=b.OrderID
group by 1,2,3
order by 1"""


##
#weather.to_sql('df_name', conn,index=False, if_exists='replace')

#https://github.com/chineseballer06/Statistical-Analysis-of-Northwind-Database/blob/master/student.ipynb

In [6]:
conn = sql.connect(db_path+db_file)
result = pd.read_sql(qry_3, conn)
conn.close()
result

,okres,rok,okres,cnt_orders,products_units
0,2016.07,2016,7,22,1462
1,2016.08,2016,8,25,1322
2,2016.09,2016,9,23,1124
3,2016.10,2016,10,26,1738
4,2016.11,2016,11,25,1735
5,2016.12,2016,12,31,2200
6,2017.01,2017,1,33,2401
7,2017.02,2017,2,29,2132
8,2017.03,2017,3,30,1770
9,2017.04,2017,4,31,1912


## Excel path

In [7]:
#xls_path=Path("D:/04_edu/11_scoring/")
#xls_full_path=os.path.join(os.getcwd( ), xls_file)
xls_path=Path(os.getcwd( ))
xls_file="py_xls_learn_v01.xlsx"
#xls_full_path=xls_path+xls_file
xls_full_path = xls_path / xls_file


In [8]:
xls_full_path

WindowsPath('D:/04_edu/11_scoring/py_xls_learn_v01.xlsx')

In [55]:
#instancje excela otwarte
xw.apps.keys()

[]

In [56]:
#wb = xw.Book(xls_full_path)
xls_app = xw.App(visible=True)
wb = xls_app.books.open(xls_full_path)
xls_app.books[0].close()  #zamykam zeszyt 1 (pusty excel)
xls_app.books

Books([<Book [py_xls_learn_v01.xlsx]>])

In [15]:
wb.sheets.count
wb.sheets[0].tables

Tables([<Table 'gb_tab1' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, <Table 'gb_tab3' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, <Table 'gb_NOWA' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, ...])

In [57]:
ark = wb.sheets[0]
ark['C23'].expand().clear() #czyści też tabelę
ark['C23'].value = [['a', 'b'], [1, 2]]
table = ark.tables.add(source=ark['C23'].expand(), name='gb_NOWA2')
ark.tables
#table

Tables([<Table 'gb_tab1' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, <Table 'gb_tab3' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, <Table 'gb_NOWA' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, ...])

In [59]:
df=result.loc[result.rok>2016,:]
table_name='gb_Nowa_update'
if table_name in [table.name for table in ark.tables]:
    ark.tables[table_name].update(df, index=False)
else:
    ark.tables.add(source=ark['H30'], name=table_name).update(df, index=False)

In [53]:
# https://peltiertech.com/structured-referencing-excel-tables/
# https://support.microsoft.com/pl-pl/office/u%C5%BCywanie-odwo%C5%82a%C5%84-strukturalnych-w-tabelach-programu-excel-f5ed2452-2337-4f71-bed3-c8ae6d2b276e

In [18]:
xw.Range('gb_Nowa_update[okres2]').value

[7.0, 8.0, 9.0, 10.0, 11.0, 12.0]

In [20]:
xw.Range('gb_Nowa_update[[#Nagłówki]; [okres2]]').value

'okres2'

In [26]:
ark.range('gb_Nowa_update[[#Nagłówki]; [okres2]]').value

'okres2'

In [38]:
ark.range('gb_Nowa_update[[#Nagłówki];[#Dane]; [cnt_orders]:[products_units]]').value
ark.range('gb_Nowa_update[[#Dane]; [cnt_orders]:[products_units]]').value

[[22.0, 1462.0],
 [25.0, 1322.0],
 [23.0, 1124.0],
 [26.0, 1738.0],
 [25.0, 1735.0],
 [31.0, 2200.0]]

In [52]:
ark.range('gb_Nowa_update[cnt_orders];gb_Nowa_update[cnt_orders]').value

[22.0, 25.0, 23.0, 26.0, 25.0, 31.0]

In [12]:
ark['C20'].value = [['a', 'b'], [1, 2], [3, 4], [5, 6]]

In [73]:
ark['B2'].top

14.25

In [83]:
chart_top    = int(ark['N30'].top)  #kotwica
chart_left   = int(ark['N30'].left) #kotwica
chart_width  = int(ark['J1'].left)  #szerokość mierzona od A1
chart_height = int(ark['A20'].top)  #wysokość mierzona od A1
#list_length  = int(ws.range('D6').value)

In [89]:
chart = ark.charts.add()
chart.set_source_data(ark.range('gb_Nowa_update[cnt_orders]'))
chart.chart_type = 'line'
chart.top = chart_top
chart.left = chart_left
chart.width = chart_width
chart.height = chart_height
chart.name='gb_wykres'
chart.api[1].SetElement(2)  # Place chart title at the top
chart.api[1].ChartTitle.Text = 'Zamówenia'  # Change text of the chart title
chart.api[1].SeriesCollection(1).XValues = '=gb_Nowa_update[[rok]:[okres2]]'

In [90]:
ark.charts

Charts([<Chart 'Wykres 1' in <Sheet [py_xls_learn_v01.xlsx]ark1>>, <Chart 'gb_wykres' in <Sheet [py_xls_learn_v01.xlsx]ark1>>])

In [91]:
wb.save()
wb.close()
xls_app.quit()

In [4]:
list_data = [10, 20, 30, 20, 15, 30, 45]
df = pd.DataFrame(list_data,columns=['kol1'])
df

,kol1
0,10
1,20
2,30
3,20
4,15
5,30
6,45


In [83]:
wb = xw.Book(xls_full_path)
ws=wb.sheets['Test']
ws['A2'].value
xw.Range('Tabela_excela[ABC]').value #kolumna z table
ws.tables.api

<win32com.gen_py.Microsoft Excel 12.0 Object Library.ListObjects instance at 0x2295217220528>

In [41]:
wb['Test'].title
ws['A4'] = 4
ws.cell(row=4, column=2, value=10)
ws.cell(row=1, column=1).value=33
ws['A1'].value


33

## old openxl

In [40]:
if os.path.isfile(xls_full_path):
    wb = ox.load_workbook(xls_full_path)
else: 
    wb = ox.Workbook()
ws=wb.active
ws.title='Test'
ws['A1']='Moj pierwszy zapis'
ws.append(['drugi wiersz']) #dodaje na końcu arkusza
ws.append(['a','b','hlkhlk'])

ws['F1']='ABC'
tab = ox.worksheet.table.Table(displayName="Tabela_excela", ref="F1:F2")
# Add a default style with striped rows and banded columns
style = ox.worksheet.table.TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=True)
tab.tableStyleInfo = style
ws.add_table(tab)

wb.save(xls_full_path)
wb.close()


writer = pd.ExcelWriter(xls_full_path,mode='a', engine='openpyxl'
                        #,if_sheet_exists="replace"
                        , if_sheet_exists="overlay" #istotne przy pracy na istniejących plikach
                       )
df.to_excel(writer, sheet_name='Test2',header=True, index=False, index_label=None, startrow=5, startcol=4)
df.to_excel(writer, sheet_name='Test2',header=True, index=False, index_label=None, startrow=7, startcol=5)
df.to_excel(writer, sheet_name='Test2',header=True, index=False, index_label=None, startrow=7, startcol=6)
df.to_excel(writer, sheet_name='Test2',header=True, index=False, index_label=None, startrow=7, startcol=7)
df.to_excel(writer, sheet_name='Test' ,header=False, index=False, index_label=None, startrow=1, startcol=5)
writer.close()

wb = ox.load_workbook(xls_full_path)
ws=wb['Test']
ws.tables['Tabela_excela'].ref="F1:F8"
# ws.tables['MyTable'].ref = "A1:E5"
# print(len(ws['F']))

wb.save(xls_full_path)
wb.close()


In [71]:
wb = ox.load_workbook(xls_full_path)
ws=wb['Test']
t={key : value for key, value in ws.tables.items()}

wb.close()  

In [72]:
t

{'Tabela_excela': 'F1:F8'}

In [5]:
basepath = xls_path
for entry in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, entry)):
        print(entry)

gb_fx_data.ipynb
gb_fx_data_vars.parquet.gzip
gb_fx_data_vars.pckl
gb_scr.ipynb
gb_scr2.ipynb
hash.ipynb
ING_1_bucketing.ipynb
ING_2_feature_selection.ipynb
ING_3_skorecard_model.ipynb
ING_benchmarks_simple.ipynb
ING_benchmark_with_EBM.ipynb
ING_using-bucketing-process.ipynb
Midgard_schedules.ipynb
MM_hyperparams.ipynb
Od zera do MLa 4.ipynb
py_excel.ipynb
py_excel_tes88.xlsx
py_excel_test10.xlsx
py_excel_test9.xlsx
rsx_new.ipynb
Udemy PD Model COMPLETE- With Comments.ipynb
Udemy Preparation - COMPLETE.ipynb
Untitled.ipynb
utils.py
xxx_dataframe_plot.py
xxx_fstrings.py
_scorecard_pipe.ipynb
_scorecard_pipe_xgb.ipynb
__py_WIKI.ipynb
